<a href="https://colab.research.google.com/github/cw00dw0rd/intro-to-knowledge-graphs/blob/master/Hacktoberfest_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!git clone -b oasis --single-branch https://github.com/cw00dw0rd/intro-to-knowledge-graphs 
!rsync -av intro-to-knowledge-graphs/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install python-arango
!pip install rdflib

In [ ]:
import json
import requests
import sys
import oasis
import time
import textwrap

from pyArango.connection import *
from arango import ArangoClient

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="Intro-Knowledge-Graph", credentialProvider="https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB")

# Connect to the temp database
# Please note that we use the python-arango driver as it has better support for ArangoSearch 
db = oasis.connect_python_arango(login)

In [ ]:
print("https://"+login["hostname"]+":"+str(login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

In [ ]:
from rdflib import Graph

In [ ]:
# Use rdflib to parse the triple to make it easier to iterate over
g = Graph()
g.parse("doyle.nt", format="nt")

aql = db.aql

# Create a new graph named "knowledge-graph" if it does not already exist.
if db.has_graph('knowledge-graph'):
    KG = db.graph('knowledge-graph')
else:
    KG = db.create_graph('knowledge-graph')

# Create the necessary collections if they don't already exist
if db.has_collection('subject'):
  pass
else:
  db.create_collection('subject',edge=False,replication_factor=3)

if db.has_collection('predicate'):
  pass
else:
  db.create_collection('predicate',edge=True,replication_factor=3)

if db.has_collection('object'):
  pass
else:
  db.create_collection('object',edge=False,replication_factor=3)

# Now that the collections have been created, create the graph edge definition 
if not KG.has_edge_definition('predicate'):
    graph = KG.create_edge_definition(
        edge_collection='predicate',
        from_vertex_collections=['subject'],
        to_vertex_collections=['object']
        )

# Iterate over the processed triple and add the documents to the graph
# We hash values here as an easy way to not worry about special characters
# Ignore errors also means if we add a document with the same subject or object 
for s, p, o in g:
  cursor = aql.execute(
  """
LET SHash = MD5(@s)
LET OHash = MD5(@o)
LET subjectHash = CONCAT('subject/', SHash)
LET objectHash = CONCAT('object/',OHash)

LET S =  (
INSERT {
    "resource": @s,
    "_key": SHash,
    "Hash": SHash
} INTO subject OPTIONS { ignoreErrors: true } RETURN NEW
)


LET O = (
INSERT {
    "resource": @o,
    "_key": OHash,
    "Hash": OHash
} INTO object OPTIONS { ignoreErrors: true } RETURN NEW
)

/* Could remove ignoreError as _key serves as duplicate check by combining 
hashes of subject and object */
INSERT {
    "resource": @p,
    "_key": CONCAT(SHash, OHash, MD5(@p)),
    "_from": subjectHash,
    "_to": objectHash
} INTO predicate OPTIONS { ignoreErrors: true }
  """,  bind_vars={'s':s, 'p': p, 'o':o}
)
# Iterate through the result cursor
for doc in cursor:
  print(doc)

If you received `Not Authorized` error, this is most likely due to the database not being quite ready yet. Attempt to re-run this code block.

In [ ]:
aql = db.aql